In [1]:
import pandas as pd
from utils import * #flatten, text_processing, prepare_dataset
from collections import Counter
import tensorflow as tf 
import os

## Data Engineering and Processing

In [4]:
data = list(pd.read_csv('severeinjury.csv', encoding = 'latin9')['text'])
text = text_processing(data)

characters = set(list(Counter(flatten(text)).keys()))
n_char = len(characters)

char2idx = dict(zip(characters, range(4, n_char + 4)))

noisy_text = get_noisy_text(text[:100], char2idx)

paddded_source_text_indexes, paddded_target_text_indexes, source_length, target_length = prepare_dataset(char2idx, text[:100], noisy_text)

ds_source = tf.data.Dataset.from_tensor_slices(paddded_source_text_indexes)
ds_target = tf.data.Dataset.from_tensor_slices(paddded_target_text_indexes)
ds_source_length = tf.data.Dataset.from_tensor_slices(source_length)
ds_target_length = tf.data.Dataset.from_tensor_slices(target_length)
dataset = tf.data.Dataset.zip((ds_source, ds_target, ds_source_length, ds_target_length))

## Model Definition

In [5]:
vector_size = 128
enc_units = 256
dec_units = 2 * enc_units

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = 'none')
encoder = Encoder(100, vector_size, enc_units)
decoder = Decoder(100, vector_size, dec_units)
optimizer = tf.keras.optimizers.Adam(1e-4)


<ZipDataset shapes: ((173,), (173,), (), ()), types: (tf.int32, tf.int32, tf.int32, tf.int32)>